In [1]:
# import packages

import os # read system path 
import csv

import matplotlib as mpl
import matplotlib.pyplot as plt

import pandas as pd
from scipy import signal
import soundfile as sf
from gudhi.point_cloud import timedelay
import numpy as np
from numpy import argmax
import math
from ripser import ripser
from persim import plot_diagrams

import umap
from sklearn.preprocessing import StandardScaler
%matplotlib qt5

from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# Path is where the voiced/voicedless wav file located
voicedPath="/Users/pfeng3/Documents/research/TopCap/revise3/LJSpeech_seg_add500/voiced/"
voicedlessPath="/Users/pfeng3/Documents/research/TopCap/revise3/LJSpeech_seg_add500/voiceless/"

In [10]:
# Do stft to phonetic data and visualize the result
for fn in os.listdir(voicedPath):
    # Read wav file as "sig"
    fileName,ext=os.path.splitext(fn)
    wavFile=voicedPath+fileName+".wav"
    sig,samplerate=sf.read(wavFile)

    # STFT 
    f, t, Zxx = signal.stft(sig, samplerate,nperseg=len(sig)/3)

    # Find the indices of the maximum magnitude for each time frame
    #max_magnitude_indices = np.argmax(np.abs(Zxx), axis=0)

    max_magnitude_indices=np.zeros(Zxx.shape[1],dtype=int)
    for i in range(Zxx.shape[1]):
        magnitude = np.abs(Zxx[:,i])
        index=np.argmax(magnitude)
        # Zero frequency is meaningless
        # Find the second largest instead 
        if f[index]==0:
            Zxx[index,i]=0
            magnitude=np.abs(Zxx[:,i])
            index=np.argmax(magnitude)
        
        max_magnitude_indices[i]=index

    # Get the corresponding frequencies
    dominant_frequencies = f[max_magnitude_indices]

    break


# Spectrogram
plt.subplot(2, 1, 1)
plt.pcolormesh(t, f, np.abs(Zxx), shading='gouraud', cmap='viridis')
plt.ylabel('Frequency (Hz)')
plt.title('STFT Spectrogram')
plt.colorbar(label='Magnitude')

# Dominant frequency over time
plt.subplot(2, 1, 2)
plt.plot(t, dominant_frequencies, marker='o', linestyle='-')
plt.xlabel('Time (s)')
plt.ylabel('Dominant Frequency (Hz)')
plt.title('Dominant Frequency over Time')
plt.tight_layout()
plt.show()

In [11]:
# Retrive features from persistent diagram 
# For voiced data
for fn in os.listdir(voicedPath):
    # Subsample dataset, retrieve 1 in 10 among dataset
    randNum=np.random.randint(10)
    if randNum !=0:
        continue

    # Read wav file as "sig"
    fileName,ext=os.path.splitext(fn)
    wavFile=voicedPath+fileName+".wav"
    sig,samplerate=sf.read(wavFile)
    
    # Write result in a csv file
    with open("STFT_Diag1.csv","a",newline="") as csvfile:
        writer=csv.writer(csvfile,quoting=csv.QUOTE_ALL)

        # STFT 
        f, t, Zxx = signal.stft(sig, samplerate,nperseg=len(sig)/3)

        # Find the indices of the maximum magnitude for each time frame
        max_magnitude_indices=np.zeros(Zxx.shape[1],dtype=int)
        for i in range(Zxx.shape[1]):
            magnitude = np.abs(Zxx[:,i])
            index=np.argmax(magnitude)
            # Zero frequency is meaningless
            # Find the second largest instead 
            if f[index]==0:
                Zxx[index,i]=0
                magnitude=np.abs(Zxx[:,i])
                index=np.argmax(magnitude)
        
            max_magnitude_indices[i]=index

        # Get the corresponding frequencies
        dominant_frequencies = f[max_magnitude_indices]

        # Drop the 11th frequency (if any), as it mainly 0 or NaN
        dominant_frequencies=dominant_frequencies[0:6]

        # Add last feature to indicate if it is voiced/ voicedless
        # 0 indicate the phone is voiced
        data=np.append(dominant_frequencies,0)
 
        writer.writerow(data)

# For voicedless data
for fn in os.listdir(voicedlessPath):
    # Subsample dataset, retrieve 1 in 10 among dataset
    randNum=np.random.randint(10)
    if randNum !=0:
        continue
    
    # Read wav file as "sig"
    fileName,ext=os.path.splitext(fn)
    wavFile=voicedlessPath+fileName+".wav"
    sig,samplerate=sf.read(wavFile)

    # Write result in a csv file
    with open("STFT_Diag1.csv","a",newline="") as csvfile:
        writer=csv.writer(csvfile)
        # STFT 
        f, t, Zxx = signal.stft(sig, samplerate,nperseg=len(sig)/3)

        # Find the indices of the maximum magnitude for each time frame
        max_magnitude_indices=np.zeros(Zxx.shape[1],dtype=int)
        for i in range(Zxx.shape[1]):
            magnitude = np.abs(Zxx[:,i])
            index=np.argmax(magnitude)
            # Zero frequency is meaningless
            # Find the second largest instead 
            if f[index]==0:
                Zxx[index,i]=0
                magnitude=np.abs(Zxx[:,i])
                index=np.argmax(magnitude)
        
            max_magnitude_indices[i]=index

        # Get the corresponding frequencies
        dominant_frequencies = f[max_magnitude_indices]

        # Drop the 11th frequency (if any), as it mainly 0 or NaN
        dominant_frequencies=dominant_frequencies[0:6]

        # Add last feature to indicate if it is voiced/ voicedless
        # 1 indicate the phone is voicedless
        data=np.append(dominant_frequencies,1)
 
        writer.writerow(data)


In [13]:
# Read the csv file into DataFrame
df=pd.read_csv('STFT_Diag1.csv', header=None)
df

,0,1,2,3,4,5,6
0,180.245232,240.326975,180.245232,180.245232,180.245232,180.245232,0.0
1,428.988327,471.887160,471.887160,171.595331,343.190661,171.595331,0.0
2,525.000000,262.500000,225.000000,262.500000,187.500000,225.000000,0.0
3,240.326975,240.326975,240.326975,240.326975,240.326975,240.326975,0.0
4,360.490463,360.490463,360.490463,360.490463,180.245232,180.245232,0.0
...,...,...,...,...,...,...,...
31367,7200.000000,7200.000000,6975.000000,6675.000000,6750.000000,7050.000000,1.0
31368,216.666667,7400.000000,6816.666667,7116.666667,7266.666667,9933.333333,1.0
31369,200.090744,180.081670,2661.206897,7003.176044,7303.312160,1880.852995,1.0
31370,180.245232,180.245232,180.245232,420.572207,2343.188011,1802.452316,1.0


In [14]:
# Shuffle 9000 samples for each voice and voiceless type

# Shuffle 9000 rows from each class
df_0 = df[df[6] == 0].sample(n=9000, random_state=42)
df_1 = df[df[6] == 1].sample(n=9000, random_state=42)

# Combine and shuffle again (optional)
df = pd.concat([df_0, df_1]).sample(frac=1, random_state=42).reset_index(drop=True)

In [15]:
# Use UMAP to reduce the feature dimension to 2
reducer = umap.UMAP()
data = df.iloc[:,0:6]
scaled_data = StandardScaler().fit_transform(data)
embedding = reducer.fit_transform(scaled_data)
embedding.shape

(18000, 2)

In [16]:
# Read the embedded feature as DataFrame
df_feature=pd.DataFrame(embedding,columns=['feature1','feature2'])
df_feature['type']=df[6]
df_feature

,feature1,feature2,type
0,13.358693,0.921169,0.0
1,14.890934,-3.633921,0.0
2,3.128443,1.133424,1.0
3,21.654802,10.001284,0.0
4,15.622794,-1.698558,0.0
...,...,...,...
17995,0.334185,-0.129739,1.0
17996,-0.910158,3.903235,1.0
17997,14.771393,2.856821,0.0
17998,19.574858,3.963336,0.0


In [17]:
# Normalization: min-max
normalized_df=(df_feature[['feature1','feature2']]-df_feature[['feature1','feature2']].min())/(df_feature[['feature1','feature2']].max()-df_feature[['feature1','feature2']].min())
normalized_df['type']=df_feature['type']
normalized_df

,feature1,feature2,type
0,0.614642,0.432155,0.0
1,0.659443,0.301064,0.0
2,0.315519,0.438264,1.0
3,0.857212,0.693472,0.0
4,0.680842,0.356762,0.0
...,...,...,...
17995,0.233818,0.401911,1.0
17996,0.197434,0.517976,1.0
17997,0.655948,0.487861,0.0
17998,0.796396,0.519706,0.0


In [ ]:
# Read/load data if needed
#normalized_df.to_pickle('stft_df_feature_nor')
#normalized_df=pd.read_pickle('stft_df_feature_nor')

In [3]:
# Count the number of samples in each class
(normalized_df['type']==0).sum()

9000

In [ ]:
# Plot the normalized result

# Set up plot configuration
SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 15

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels

# Group the data based on voiced/ voicedless
groups = normalized_df.groupby('type')

# Plot
fig, ax = plt.subplots(figsize=(6, 6))
ax.margins(0.05)
typeDict= {1:'voiceless',0:'voiced'}
for type, group in groups:
    if type==1:
        ax.plot(group.feature1, group.feature2, marker='o', linestyle='', ms=2, label=typeDict[type],alpha=0.5, color='#4d4dff')
    if type==0:
        ax.plot(group.feature1, group.feature2, marker='o', linestyle='', ms=2, label=typeDict[type],alpha=0.5, color='#ff5c33')
legend=ax.legend(fontsize=15,markerscale=4,loc='lower left')
plt.xlabel('UMAP_1',fontsize=15)
plt.ylabel('UMAP_2',fontsize=15)

## Save figure as pdf file
#plt.savefig("/Users/pfeng3/Documents/research/TopCap/picture/featureAna_stft1.pdf", format="pdf", bbox_inches="tight")

In [ ]:
# Plot individual 
plt.figure(figsize=(6, 3))

for type, group in groups:
    if type==0:
        plt.subplot(1, 2, 1)
        plt.plot(group.feature1, group.feature2, marker='o', linestyle='', ms=2, label=typeDict[type], alpha=0.5, color='#ff5c33')
        plt.legend(['voiced'],fontsize=10,markerscale=4,loc='lower left')
        plt.xlabel('UMAP_1')
        plt.ylabel('UMAP_2')
        plt.xlim([-0.1,1.1])
        plt.ylim([-0.1,1.1])
        plt
    if type==1:
        plt.subplot(1, 2, 2)
        plt.plot(group.feature1, group.feature2, marker='o', linestyle='', ms=2, label=typeDict[type], alpha=0.5, color='#4d4dff')
        plt.legend(['voiceless'],fontsize=10,markerscale=4,loc='lower left')
        plt.xlabel('UMAP_1')
        plt.ylabel('UMAP_2')
        plt.xlim([-0.1,1.1])
        plt.ylim([-0.1,1.1])

plt.tight_layout()
plt.show()

## Save figure as pdf file
#plt.savefig("/Users/pfeng3/Documents/research/TopCap/picture/featureAna_stft2.pdf", format="pdf", bbox_inches="tight")

In [29]:
x_train, x_test, y_train, y_test = train_test_split(normalized_df[['feature1','feature2']], normalized_df['type'], test_size=0.2, random_state=42)

In [31]:
# Classification model knn 

knn = KNeighborsClassifier(n_neighbors=20) # Set the number of neighbors
knn.fit(x_train, y_train)

y_pred = knn.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8447222222222223


In [25]:
# Train a linear model to classify the data, and plot the decision boundary

# Create a RidgeClassifier object
ridge = RidgeClassifier(alpha=1.0)

# Fit the model to the training data
ridge.fit(x_train, y_train.values.ravel())

# Predict the labels for the test data
y_pred = ridge.predict(x_test)

# Evaluate the performance of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8844444444444445


In [30]:
# Logistic classification
from sklearn.linear_model import LogisticRegression

# Create a logistic regression model
model = LogisticRegression()

# Train the model using the training data
model.fit(x_train, y_train.values.ravel())

# Predict class labels for the test data
y_pred = model.predict(x_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.8833333333333333


In [32]:
# SVM classification
from sklearn import svm

# Create an SVM classifier
clf = svm.SVC(kernel='linear')
# Train the classifier
clf.fit(x_train, y_train.values.ravel())
# Predict on the test set
y_pred = clf.predict(x_test)
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7891666666666667


In [33]:
# LDA model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

clf = LinearDiscriminantAnalysis()
# Train the classifier
clf.fit(x_train, y_train.values.ravel())
# Predict on the test set
y_pred = clf.predict(x_test)
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7897222222222222


In [10]:
# compute statistical information from data

normalized_df['feature1'].mean()
normalized_df['feature1'].std()

0.2291782796382904

In [12]:
normalized_df['feature2'].mean()
normalized_df['feature2'].std()

0.12391874194145203

In [13]:
nor_voice_df=normalized_df[normalized_df['type']==0]
print(f'Mean for voiced data: {nor_voice_df.mean()}')
print(f'Std for voiced data: {nor_voice_df.std()}')

Mean for voiced data: feature1    0.699551
feature2    0.521323
type        0.000000
dtype: float64
Std for voiced data: feature1    0.105476
feature2    0.103453
type        0.000000
dtype: float64


In [14]:
nor_voiceless_df=normalized_df[normalized_df['type']==1]
print(f'Mean for voiceless data: {nor_voiceless_df.mean()}')
print(f'Std for voiceless data: {nor_voiceless_df.std()}')

Mean for voiceless data: feature1    0.31242
feature2    0.44649
type        1.00000
dtype: float64
Std for voiceless data: feature1    0.145592
feature2    0.134606
type        0.000000
dtype: float64
